In [2]:
import duckdb
import pandas as pd
import sqlalchemy

# Import ipython-sql Jupyter extension to create SQL cells
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [3]:
#Set configrations on ipython-sql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [4]:
#Connect ipython-sql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

In [28]:
%%sql df_attributes <<

WITH stg_person AS (
  SELECT 
    PERSON_ID AS id,
    GENDER_CONCEPT_ID,
	date_part('year', current_date) - YEAR_OF_BIRTH AS age,
    -- RACE_CONCEPT_IDs are not in Eunomia's CONCEPT table
    RACE_SOURCE_VALUE AS race
  FROM read_csv_auto('../data/eunomia/PERSON.csv')
),

mapped_person AS (
	SELECT
		id,
		LOWER(CONCEPT_NAME) AS gender,
		age,
		race
	FROM stg_person
	LEFT JOIN read_csv_auto('../data/eunomia/CONCEPT.csv') AS concept
	ON stg_person.GENDER_CONCEPT_ID = concept.CONCEPT_ID
	ORDER BY id ASC
)

SELECT
	id,
	'gender' AS variable,
	gender AS value,
	'string' AS type
FROM mapped_person
UNION ALL
SELECT
	id,
	'age' AS variable,
	age AS value,
	'int' AS type
FROM mapped_person
UNION ALL
SELECT
	id,
	'race' AS variable,
	race AS value,
	'string' AS type
FROM mapped_person
ORDER BY id, variable


Returning data to local variable df_attributes


In [29]:
df_attributes

,id,variable,value,type
0,1,age,73,int
1,1,gender,male,string
2,1,race,white,string
3,2,age,102,int
4,2,gender,female,string
...,...,...,...,...
8077,5337,gender,male,string
8078,5337,race,white,string
8079,5343,age,39,int
8080,5343,gender,female,string


In [42]:
%%sql df_events <<
WITH stg_condition AS (
  SELECT 
    PERSON_ID AS id,
    CONDITION_CONCEPT_ID,
    CAST(CONDITION_START_DATE AS DATETIME) + INTERVAL 1 SECOND AS date
  FROM read_csv_auto('../data/eunomia/CONDITION_OCCURRENCE.csv')
)

SELECT
  stg_condition.id,
  stg_condition.date,
  concept.VOCABULARY_ID AS codeclass,
  concept.CONCEPT_CODE AS code
FROM stg_condition
LEFT JOIN read_csv_auto('../data/eunomia/CONCEPT.csv') AS concept
ON stg_condition.CONDITION_CONCEPT_ID = concept.CONCEPT_ID
ORDER BY id ASC, date ASC

Returning data to local variable df_events


In [43]:
df_events

,id,date,codeclass,code
0,1,1953-02-06 00:00:01,SNOMED,43878008
1,1,1958-03-11 00:00:01,SNOMED,65966004
2,1,1961-07-05 00:00:01,SNOMED,195662009
3,1,1967-05-30 00:00:01,SNOMED,444814009
4,1,1970-12-03 00:00:01,SNOMED,44465007
...,...,...,...,...
65327,5343,2010-04-01 00:00:01,SNOMED,195662009
65328,5343,2015-02-19 00:00:01,SNOMED,195662009
65329,5343,2016-11-04 00:00:01,SNOMED,284549007
65330,5343,2017-06-23 00:00:01,SNOMED,195662009


In [44]:
df_attributes.to_csv('../out/jupyter/attributes.csv', index=False)
df_events.to_csv('../out/jupyter/events.csv', index=False)